In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df = pd.read_csv('/kaggle/input/amazon-ratings/ratings_Beauty.csv')

In [3]:
df.drop('Timestamp', axis=1, inplace=True)
df.shape

(2023070, 3)

In [4]:
df.head()

,UserId,ProductId,Rating
0,A39HTATAQ9V7YF,0205616461,5.0
1,A3JM6GV9MNOF9X,0558925278,3.0
2,A1Z513UWSAAO0F,0558925278,5.0
3,A1WMRR494NWEWV,0733001998,4.0
4,A3IAAVS479H7M7,0737104473,1.0


In [5]:
# change the id of users with incremental numbers which is easy to deal with
user_ids = df['UserId'].unique()
user_id_map = {old_id: new_id for new_id, old_id in enumerate(user_ids)}
df['UserId'] = df['UserId'].map(user_id_map)

# change the id of products with incremental numbers which is easy to deal with
product_ids = df['ProductId'].unique()
product_id_map = {old_id: new_id for new_id, old_id in enumerate(product_ids)}
df['ProductId'] = df['ProductId'].map(product_id_map)

In [6]:
df.head()

,UserId,ProductId,Rating
0,0,0,5.0
1,1,1,3.0
2,2,1,5.0
3,3,2,4.0
4,4,3,1.0


# Some variables to be used around the notebook

In [7]:
# get all unique products
allProducts = df['ProductId'].unique()

# Start developing the model

In [8]:
def build_model(num_users, num_products, embedding_size):
    user_input = Input(shape=(), name='user_input')
    product_input = Input(shape=(), name='product_input')
    
    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embeddings')(user_input)
    product_embedding = Embedding(input_dim=num_products, output_dim=embedding_size, name='product_embeddings')(product_input)
    
    dot_product = Dot(axes=1, name='dot_product')([user_embedding, product_embedding])
    flatten = Flatten(name='flatten_layer')(dot_product)
    
    model = Model(inputs=[user_input, product_input], outputs=flatten)
    
    return model

In [9]:
num_users = len(df['UserId'].unique())
num_products = len(df['ProductId'].unique())
embedding_size = 50
# print(num_users, num_products)

model = build_model(num_users, num_products, embedding_size)

In [10]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 product_input (InputLayer)     [(None,)]            0           []                               
                                                                                                  
 user_embeddings (Embedding)    (None, 50)           60513550    ['user_input[0][0]']             
                                                                                                  
 product_embeddings (Embedding)  (None, 50)          12463700    ['product_input[0][0]']          
                                                                                              

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df[['UserId', 'ProductId']], df[['Rating']], test_size=0.1)

In [12]:
model.fit(
    x=[x_train['UserId'], x_train['ProductId']],
    y=y_train['Rating'],
    batch_size=64,
    epochs=10,
    validation_split=0.1
)

Epoch 1/10
25605/25605 [==============================] - 266s 10ms/step - loss: 18.8227 - accuracy: 1.6904e-04 - val_loss: 18.5742 - val_accuracy: 4.9979e-04
Epoch 2/10
25605/25605 [==============================] - 260s 10ms/step - loss: 15.3845 - accuracy: 0.0207 - val_loss: 17.9218 - val_accuracy: 0.0055
Epoch 3/10
25605/25605 [==============================] - 262s 10ms/step - loss: 8.5447 - accuracy: 0.0693 - val_loss: 17.7121 - val_accuracy: 0.0096
Epoch 4/10
25605/25605 [==============================] - 263s 10ms/step - loss: 4.3510 - accuracy: 0.0829 - val_loss: 17.7391 - val_accuracy: 0.0105
Epoch 5/10
25605/25605 [==============================] - 265s 10ms/step - loss: 2.5791 - accuracy: 0.0663 - val_loss: 17.7468 - val_accuracy: 0.0111
Epoch 6/10
25605/25605 [==============================] - 263s 10ms/step - loss: 1.5716 - accuracy: 0.0803 - val_loss: 17.7955 - val_accuracy: 0.0111
Epoch 7/10
25605/25605 [==============================] - 263s 10ms/step - loss: 1.1111 - 

In [13]:
test_loss = model.evaluate(
    x=[x_test['UserId'], x_test['ProductId']],
    y=y_test,
    batch_size=64
)
print(f'Test Loss: {test_loss}')

3162/3162 [==============================] - 7s 2ms/step - loss: 17.7842 - accuracy: 0.0121
Test Loss: [17.78415298461914, 0.01205593440681696]


In [14]:
print("Number of Users:      " + str(num_users))
print("Number of Products:    " + str(num_products))

Number of Users:      1210271
Number of Products:    249274


### Save the model

In [15]:
# Save the model after training
model.save('collaborative_filtering_model.h5')

In [16]:
# Load the model if you shutted down the notebook
model = tf.keras.models.load_model('collaborative_filtering_model.h5')

## Predicting the best products for a specific user

In [17]:
def recommend_product_for_user(idOfUser):
    # get the data for a specifc user
    data = df[df['UserId'] == idOfUser].reset_index().drop(['index', 'Rating'], axis=1)
    
    # make a DataFrame with all products for a specific user
    allProdcutsPerUser = pd.DataFrame({'UserId': idOfUser, 'ProductId': allProducts.copy()})
    print(allProdcutsPerUser)

In [18]:
datPerUser = recommend_product_for_user(4)

        UserId  ProductId
0            4          0
1            4          1
2            4          2
3            4          3
4            4          4
...        ...        ...
249269       4     249269
249270       4     249270
249271       4     249271
249272       4     249272
249273       4     249273

[249274 rows x 2 columns]
